<a href="https://colab.research.google.com/github/Forpee/NLP-practice/blob/master/text_generation/draft_tf_tut_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text Generation with rnn



1. Get data
2. Read data
3. Split text into chars
4. Convert chars to id's
5. Create a way to get text from id's
6. Create dataset from id's
7. Batch dataset
8. Shift all chars in dataset by one to get labels
9. Improve performance on dataset
10. Build the model



Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. 

This makes the task a classification problem.
Which char (class) has the highest probability of being next

#Imports

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

#Get data

In [2]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


#Examine data

In [3]:
text= open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'{len(text)}')

1115394


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


#Process the text

In [6]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

To convert from numbers back to chars

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

You can tf.strings.reduce_join to join the characters back into strings.

In [9]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [10]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [13]:
seq_length = 100

The batch method lets you easily convert these individual characters to sequences of the desired size.

In [14]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [15]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [17]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [18]:
dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


#Create training batches

In [20]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

#Build the model

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding=tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru=tf.keras.layers.GRU(rnn_units,
                                 return_sequences=True,
                                 return_state=True)
    self.dense=tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x=inputs
    x=self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states,training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

We need to return the previous states so the model has some context as to what to predict next

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [25]:
model.compile(optimizer='adam', loss=loss)

checkpoints

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS = 20

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 24s 54ms/step - loss: 2.7401
Epoch 2/20
172/172 [==============================] - 10s 53ms/step - loss: 2.0071
Epoch 3/20
172/172 [==============================] - 10s 54ms/step - loss: 1.7314
Epoch 4/20
172/172 [==============================] - 11s 55ms/step - loss: 1.5700
Epoch 5/20
172/172 [==============================] - 11s 58ms/step - loss: 1.4684
Epoch 6/20
172/172 [==============================] - 11s 59ms/step - loss: 1.3982
Epoch 7/20
172/172 [==============================] - 11s 59ms/step - loss: 1.3443
Epoch 8/20
172/172 [==============================] - 11s 59ms/step - loss: 1.2970
Epoch 9/20
172/172 [==============================] - 11s 56ms/step - loss: 1.2568
Epoch 10/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2177
Epoch 11/20
172/172 [==============================] - 11s 59ms/step - loss: 1.1784
Epoch 12/20
172/172 [==============================] - 12s 60ms/step - loss: 1.1385
E

In [29]:
inputs=tf.constant(['ROMEO:'])

In [30]:
input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
input_ids = ids_from_chars(input_chars).to_tensor()

In [31]:
predicted_logits, states=model(inputs=input_ids, return_state=True)

In [32]:
predicted_logits, states

(<tf.Tensor: shape=(1, 6, 66), dtype=float32, numpy=
 array([[[-1.7266843e+00,  1.6664969e+00,  1.3841555e+00, -3.0975657e+00,
          -1.7980670e+00, -1.4096066e+00, -1.1399949e+00, -1.3060118e-01,
          -3.3748140e+00, -1.1185836e+00, -6.7566937e-01,  4.2574306e+00,
           1.0946325e+00, -1.4821867e+00,  3.9027307e+00,  1.0068525e+00,
           4.4673519e+00,  4.6906734e+00,  4.6421552e+00, -2.1777422e+00,
           2.3919015e+00, -2.0973432e+00,  5.6303878e+00,  2.6336285e-01,
           3.3451359e+00,  1.5621504e+00,  1.2552382e+00,  4.2832232e-01,
           5.1199045e+00,  3.4916736e-02, -2.9156545e-01, -1.4514128e+00,
           1.4173747e+00,  2.9178884e+00,  4.8879576e+00, -1.7767884e-01,
           4.3218193e+00, -1.0473108e+00,  4.0053306e+00, -2.2250435e-01,
           2.4680347e+00, -3.9122834e+00, -1.0775247e+00, -1.6967285e+00,
           3.3886709e+00, -6.4940829e+00, -2.8806665e+00, -2.5184665e+00,
           4.3427043e+00, -2.3022864e+00,  8.0149883e-01, -

In [34]:
# Only use the last prediction.
predicted_logits = predicted_logits[:, -1, :]